[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/sdxl-turbo-colab/blob/main/sdxl_turbo_colab.ipynb)

In [ ]:
%cd /content
!git clone -b turbo https://github.com/camenduru/generative-models
%cd /content/generative-models

!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/stabilityai/sdxl-turbo/resolve/main/sd_xl_turbo_1.0_fp16.safetensors -d /content/generative-models/checkpoints -o sd_xl_turbo_1.0.safetensors

!pip install -q streamlit einops invisible-watermark omegaconf pytorch_lightning kornia open-clip-torch streamlit-keyup
!pip install -q https://download.pytorch.org/whl/cu118/xformers-0.0.22.post4%2Bcu118-cp310-cp310-manylinux2014_x86_64.whl
!pip install -q git+https://github.com/openai/CLIP.git
!pip install -q -e .

import os
import shlex
import subprocess
from pathlib import Path
from typing import Union

id_rsa_file = "/content/id_rsa"
id_rsa_pub_file = "/content/id_rsa.pub"
if os.path.exists(id_rsa_file):
    os.remove(id_rsa_file)
if os.path.exists(id_rsa_pub_file):
    os.remove(id_rsa_pub_file)

def gen_key(path: Union[str, Path]) -> None:
    path = Path(path)
    arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
    args = shlex.split(arg_string)
    subprocess.run(args, check=True)
    path.chmod(0o600)

gen_key(id_rsa_file)

import threading
def tunnel():
  !ssh -R 80:127.0.0.1:8501 -o StrictHostKeyChecking=no -i /content/id_rsa remote.moe
threading.Thread(target=tunnel, daemon=True).start()

!streamlit run /content/generative-models/scripts/demo/turbo.py